In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
import os
import multiprocessing
from multiprocessing.pool import ThreadPool

import numpy as np
import pandas as pd
import datetime

import matplotlib.pyplot as plt
import tqdm
import ipywidgets as widgets
from ipywidgets import interact, interact_manual

In [3]:
from data_io import load_data_by_ticker, get_call_data_for_given_ticker, display_all, display_shape, calls_amount

In [4]:
import matplotlib.font_manager as font_manager

font_dirs = ['/usr/share/fonts/truetype/nanum']
font_files = font_manager.findSystemFonts(fontpaths=font_dirs)
font_list = font_manager.createFontList(font_files)
font_manager.fontManager.ttflist.extend(font_list)

plt.rcParams['font.family'] = 'NanumGothic'

In [5]:
calls = pd.read_pickle('data/CALLS_TRANSFORMED.pkl')
calls[0:1]

,월,요일,평균기온,최고기온,최저기온,일강수량,평균풍속,최다풍향,평균상대습도,가조시간,...,"중랑구, 20대, 남","중랑구, 20대, 여","중랑구, 30대, 남","중랑구, 30대, 여","중랑구, 40대, 남","중랑구, 40대, 여","중랑구, 50대, 남","중랑구, 50대, 여","중랑구, 60대, 남","중랑구, 60대, 여"
2013-08-01,8,목요일,28.0,24.5,32.1,0.0,1.7,50.0,67.3,14.1,...,79,91,98,133,62,88,33.0,55,NaN,6.0


In [6]:
calls_original = pd.read_pickle('data/CALLS_CLIMATE.pkl')
calls_original[0:1]

,날짜,시군구,성별,연령대,총건수,월,요일,평균기온,최고기온,최저기온,...,최다풍향,평균상대습도,가조시간,일조시간,강설량,평균운량,미세먼지,공휴일_당일,공휴일_전날,일
0,2013-08-01,강남구,남,10대,71,8,목요일,28.0,24.5,32.1,...,50.0,67.3,14.1,6.0,0.0,7.1,34,0,0,1


In [7]:
calls_all = pd.read_pickle('data/CALLS_ALL_TRANSFORMED.pkl')
calls_all = calls_all.groupby(['업종', '일자'])['총건수'].sum()
calls_all = calls_all.to_frame().reset_index(drop=False)
calls_all.columns = ['날짜' if x=='일자' else x for x in calls_all.columns]
display(calls_all)
calls_all['업종'].unique()

,업종,날짜,총건수
0,족발/보쌈전문,2013-08-01,2989
1,족발/보쌈전문,2013-08-02,3389
2,족발/보쌈전문,2013-08-03,3935
3,족발/보쌈전문,2013-08-04,3682
4,족발/보쌈전문,2013-08-05,2645
...,...,...,...
8883,피자,2019-08-27,3832
8884,피자,2019-08-28,3974
8885,피자,2019-08-29,4543
8886,피자,2019-08-30,5118


array(['족발/보쌈전문', '중국음식', '치킨', '피자'], dtype=object)

In [8]:
calls_all.shape

(8888, 3)

In [9]:
calls.index

DatetimeIndex(['2013-08-01', '2013-08-02', '2013-08-03', '2013-08-04',
               '2013-08-05', '2013-08-06', '2013-08-07', '2013-08-08',
               '2013-08-09', '2013-08-10',
               ...
               '2019-08-22', '2019-08-23', '2019-08-24', '2019-08-25',
               '2019-08-26', '2019-08-27', '2019-08-28', '2019-08-29',
               '2019-08-30', '2019-08-31'],
              dtype='datetime64[ns]', length=2222, freq=None)

In [10]:
for food in calls_all['업종'].unique() :
    print(food)
    target = calls_all[calls_all['업종']==food]
    target.index = target['날짜']
    target = target.drop(['날짜', '업종'], axis=1)
    calls[food] = target['총건수']
    display(target)

족발/보쌈전문


,총건수
날짜,
2013-08-01,2989
2013-08-02,3389
2013-08-03,3935
2013-08-04,3682
2013-08-05,2645
...,...
2019-08-27,2359
2019-08-28,2341
2019-08-29,2612


중국음식


,총건수
날짜,
2013-08-01,29310
2013-08-02,30199
2013-08-03,35965
2013-08-04,36219
2013-08-05,27561
...,...
2019-08-27,10162
2019-08-28,10491
2019-08-29,14513


치킨


,총건수
날짜,
2013-08-01,21906
2013-08-02,27457
2013-08-03,32984
2013-08-04,31856
2013-08-05,19661
...,...
2019-08-27,8168
2019-08-28,8694
2019-08-29,9406


피자


,총건수
날짜,
2013-08-01,6624
2013-08-02,8091
2013-08-03,9601
2013-08-04,10519
2013-08-05,6318
...,...
2019-08-27,3832
2019-08-28,3974
2019-08-29,4543


In [11]:
calls.to_pickle('data/CALLS_ALL_MERGED.pkl')

In [12]:
display(calls)

,월,요일,평균기온,최고기온,최저기온,일강수량,평균풍속,최다풍향,평균상대습도,가조시간,...,"중랑구, 40대, 남","중랑구, 40대, 여","중랑구, 50대, 남","중랑구, 50대, 여","중랑구, 60대, 남","중랑구, 60대, 여",족발/보쌈전문,중국음식,치킨,피자
2013-08-01,8,목요일,28.0,24.5,32.1,0.0,1.7,50.0,67.3,14.1,...,62,88,33.0,55,NaN,6.0,2989,29310,21906,6624
2013-08-02,8,금요일,27.4,25.2,29.3,7.5,2.0,50.0,80.4,14.1,...,82,101,33.0,44,10.0,18.0,3389,30199,27457,8091
2013-08-03,8,토요일,28.9,26.7,32.6,0.1,3.7,230.0,72.3,14.0,...,108,136,36.0,48,17.0,19.0,3935,35965,32984,9601
2013-08-04,8,일요일,28.3,26.4,31.3,4.5,2.4,250.0,76.4,14.0,...,98,120,37.0,46,16.0,14.0,3682,36219,31856,10519
2013-08-05,8,월요일,27.7,26.4,30.1,7.0,3.2,230.0,78.9,14.0,...,56,65,30.0,28,8.0,12.0,2645,27561,19661,6318
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-08-27,8,화요일,25.7,22.2,29.3,20.0,1.0,290.0,71.1,13.2,...,25,44,12.0,8,8.0,10.0,2359,10162,8168,3832
2019-08-28,8,수요일,26.1,23.6,30.2,0.0,1.9,270.0,66.2,13.2,...,20,39,17.0,21,10.0,5.0,2341,10491,8694,3974
2019-08-29,8,목요일,23.4,20.1,26.4,36.9,2.2,160.0,77.1,13.1,...,24,37,9.0,18,11.0,10.0,2612,14513,9406,4543
2019-08-30,8,금요일,22.6,19.5,27.0,0.2,2.4,200.0,66.3,13.1,...,37,51,14.0,25,10.0,8.0,2959,11548,12604,5118


In [13]:
kospi_downloaded = pd.read_pickle('history/kospi_download_result.pkl')
kospi_downloaded[0:1]

,종목코드,상장일,결과,기업명,업종
0,095570,2015-08-21,Naver,AJ네트웍스,산업용 기계 및 장비 임대업


In [14]:
kospi = pd.read_csv('metadata/kospi.csv', error_bad_lines=False)
kospi[0:1]

b'Skipping line 437: expected 12 fields, saw 13\n'


,번호,종목코드,기업명,업종코드,업종,상장주식수(주),자본금(원),액면가(원),통화구분,대표전화,주소,총카운트
0,1,095570,AJ네트웍스,147603.0,산업용 기계 및 장비 임대업,"46,822,295","46,822,295,000","1,000",원(KRW),02-6363-9999,"서울특별시 송파구 정의로8길 9 (문정동,AJ빌딩)",789.0


In [17]:
kosis_ecommerce_2013_2017 = pd.read_csv('data/kosis_ecommerce(201301-201712).csv', index_col=[0,1])
kosis_ecommerce_2013_2017 = kosis_ecommerce_2013_2017.reset_index(drop=False)
kosis_ecommerce_2013_2017['상품군별'] = kosis_ecommerce_2013_2017['상품군별'].fillna(method='ffill')
kosis_ecommerce_2013_2017['상품군별'] = kosis_ecommerce_2013_2017['상품군별'].apply(lambda x : x.replace('\u3000', '').replace(' ', ''))
display(kosis_ecommerce_2013_2017.T[kosis_ecommerce_2013_2017.T.columns[:3]])

,0,1,2
상품군별,합계,합계,합계
판매매체별,합계,인터넷쇼핑,모바일쇼핑
2013. 01,"3,189,033","2,815,624","373,409"
2013. 02,"2,752,824","2,405,536","347,288"
2013. 03,"3,033,675","2,626,963","406,712"
...,...,...,...
2017. 08,"6,675,062","2,586,174","4,088,888"
2017. 09,"6,905,823","2,704,518","4,201,306"
2017. 10,"6,507,896","2,444,311","4,063,585"
2017. 11,"7,693,426","2,928,876","4,764,550"


In [19]:
kosis_ecommerce_2017_2019 = pd.read_csv('data/kosis_ecommerce(201701-201909).csv', index_col=[0,1])
kosis_ecommerce_2017_2019 = kosis_ecommerce_2017_2019.reset_index(drop=False)
kosis_ecommerce_2017_2019['상품군별'] = kosis_ecommerce_2017_2019['상품군별'].fillna(method='ffill')
kosis_ecommerce_2017_2019['상품군별'] = kosis_ecommerce_2017_2019['상품군별'].apply(lambda x : x.replace(' ', ''))
kosis_ecommerce_2017_2019['판매매체별'] = kosis_ecommerce_2017_2019['판매매체별'].apply(lambda x : '합계' if x=='계' else x)
display(kosis_ecommerce_2017_2019.T[kosis_ecommerce_2017_2019.T.columns[:3]])

,0,1,2
상품군별,합계,합계,합계
판매매체별,합계,인터넷쇼핑,모바일쇼핑
2017. 01,"7,310,479","3,390,183","3,920,295"
2017. 02,"7,148,849","3,298,823","3,850,025"
2017. 03,"7,747,011","3,585,449","4,161,562"
2017. 04,"7,380,795","3,304,819","4,075,976"
2017. 05,"7,621,719","3,359,693","4,262,026"
2017. 06,"7,515,130","3,341,138","4,173,992"
2017. 07,"7,898,554","3,404,599","4,493,955"
2017. 08,"7,859,658","3,395,357","4,464,301"


In [84]:
print(kosis_ecommerce_2013_2017['상품군별'].unique())
display_all(kosis_ecommerce_2013_2017[['상품군별', '판매매체별']])

['합계' '컴퓨터및주변기기' '가전·전자·통신기기' '소프트웨어' '서적' '사무·문구' '음반·비디오·악기'
 '의류·패션및관련상품' '의복' '신발' '가방' '패션용품및악세사리' '스포츠·레저용품' '화장품' '아동·유아용품'
 '음·식료품' '농축수산물' '생활·자동차용품' '가구' '꽃' '여행및예약서비스' '각종서비스' '기타' '애완용품']


,상품군별,판매매체별
0,합계,합계
1,합계,인터넷쇼핑
2,합계,모바일쇼핑
3,컴퓨터및주변기기,합계
4,컴퓨터및주변기기,인터넷쇼핑
5,컴퓨터및주변기기,모바일쇼핑
6,가전·전자·통신기기,합계
7,가전·전자·통신기기,인터넷쇼핑
8,가전·전자·통신기기,모바일쇼핑
9,소프트웨어,합계


In [80]:
print(kosis_ecommerce_2017_2019['상품군별'].unique())
display_all(kosis_ecommerce_2017_2019[['상품군별', '판매매체별']])

['합계' '컴퓨터 및 주변기기' '가전·전자·통신기기' '서적' '사무·문구' '의복' '신발' '가방' '패션용품 및 액세서리'
 '스포츠·레저용품' '화장품' '아동·유아용품' '음·식료품' '농축수산물' '생활용품' '자동차용품' '가구' '애완용품'
 '여행 및 교통서비스' '문화 및 레저서비스' 'e쿠폰서비스' '음식서비스' '기타서비스' '기타']


,상품군별,판매매체별
0,합계,합계
1,합계,인터넷쇼핑
2,합계,모바일쇼핑
3,컴퓨터 및 주변기기,합계
4,컴퓨터 및 주변기기,인터넷쇼핑
5,컴퓨터 및 주변기기,모바일쇼핑
6,가전·전자·통신기기,합계
7,가전·전자·통신기기,인터넷쇼핑
8,가전·전자·통신기기,모바일쇼핑
9,서적,합계


In [20]:
temp_2013 = kosis_ecommerce_2013_2017.T[kosis_ecommerce_2013_2017.T.columns[:3]]
temp_2017 = kosis_ecommerce_2017_2019.T[kosis_ecommerce_2017_2019.T.columns[:3]]

In [34]:
temp_2013.columns = ['합계', '인터넷쇼핑', '모바일쇼핑']
temp_2017.columns = ['합계', '인터넷쇼핑', '모바일쇼핑']
temp = pd.concat([temp_2013.iloc[2:], temp_2017.iloc[2:]], ignore_index=False)
temp.index = [x.replace(' p)', '') for x in temp.index]
temp.index = pd.to_datetime(temp.index)
temp.to_pickle('kosis_ecommerce_all.pkl')
temp

,합계,인터넷쇼핑,모바일쇼핑
2013-01-01,"3,189,033","2,815,624","373,409"
2013-02-01,"2,752,824","2,405,536","347,288"
2013-03-01,"3,033,675","2,626,963","406,712"
2013-04-01,"3,014,762","2,606,326","408,436"
2013-05-01,"3,170,619","2,718,382","452,237"
...,...,...,...
2019-05-01,"11,232,072","4,078,119","7,153,953"
2019-06-01,"10,557,017","3,702,233","6,854,783"
2019-07-01,"11,185,384","3,967,022","7,218,362"
2019-08-01,"11,190,709","3,943,856","7,246,854"


In [37]:
calls_all_merged = pd.read_pickle('data/CALLS_ALL_MERGED.pkl')
calls_all_merged

,월,요일,평균기온,최고기온,최저기온,일강수량,평균풍속,최다풍향,평균상대습도,가조시간,...,"중랑구, 40대, 남","중랑구, 40대, 여","중랑구, 50대, 남","중랑구, 50대, 여","중랑구, 60대, 남","중랑구, 60대, 여",족발/보쌈전문,중국음식,치킨,피자
2013-08-01,8,목요일,28.0,24.5,32.1,0.0,1.7,50.0,67.3,14.1,...,62,88,33.0,55,NaN,6.0,2989,29310,21906,6624
2013-08-02,8,금요일,27.4,25.2,29.3,7.5,2.0,50.0,80.4,14.1,...,82,101,33.0,44,10.0,18.0,3389,30199,27457,8091
2013-08-03,8,토요일,28.9,26.7,32.6,0.1,3.7,230.0,72.3,14.0,...,108,136,36.0,48,17.0,19.0,3935,35965,32984,9601
2013-08-04,8,일요일,28.3,26.4,31.3,4.5,2.4,250.0,76.4,14.0,...,98,120,37.0,46,16.0,14.0,3682,36219,31856,10519
2013-08-05,8,월요일,27.7,26.4,30.1,7.0,3.2,230.0,78.9,14.0,...,56,65,30.0,28,8.0,12.0,2645,27561,19661,6318
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-08-27,8,화요일,25.7,22.2,29.3,20.0,1.0,290.0,71.1,13.2,...,25,44,12.0,8,8.0,10.0,2359,10162,8168,3832
2019-08-28,8,수요일,26.1,23.6,30.2,0.0,1.9,270.0,66.2,13.2,...,20,39,17.0,21,10.0,5.0,2341,10491,8694,3974
2019-08-29,8,목요일,23.4,20.1,26.4,36.9,2.2,160.0,77.1,13.1,...,24,37,9.0,18,11.0,10.0,2612,14513,9406,4543
2019-08-30,8,금요일,22.6,19.5,27.0,0.2,2.4,200.0,66.3,13.1,...,37,51,14.0,25,10.0,8.0,2959,11548,12604,5118


In [36]:
temp

,합계,인터넷쇼핑,모바일쇼핑
2013-01-01,"3,189,033","2,815,624","373,409"
2013-02-01,"2,752,824","2,405,536","347,288"
2013-03-01,"3,033,675","2,626,963","406,712"
2013-04-01,"3,014,762","2,606,326","408,436"
2013-05-01,"3,170,619","2,718,382","452,237"
...,...,...,...
2019-05-01,"11,232,072","4,078,119","7,153,953"
2019-06-01,"10,557,017","3,702,233","6,854,783"
2019-07-01,"11,185,384","3,967,022","7,218,362"
2019-08-01,"11,190,709","3,943,856","7,246,854"


In [46]:
merge_test = calls_all_merged.merge(temp, how='outer', left_on=calls_all_merged.index, right_on=temp.index)
merge_test = merge_test[merge_test['월'].notnull()]
merge_test.index = merge_test['key_0']
merge_test = merge_test.drop(['key_0'], axis=1)
for column in temp.columns : 
    merge_test[column] = merge_test[column].fillna(method='ffill')

merge_test

,월,요일,평균기온,최고기온,최저기온,일강수량,평균풍속,최다풍향,평균상대습도,가조시간,...,"중랑구, 50대, 여","중랑구, 60대, 남","중랑구, 60대, 여",족발/보쌈전문,중국음식,치킨,피자,합계,인터넷쇼핑,모바일쇼핑
key_0,,,,,,,,,,,,,,,,,,,,,
2013-08-01,8.0,목요일,28.0,24.5,32.1,0.0,1.7,50.0,67.3,14.1,...,55.0,NaN,6.0,2989.0,29310.0,21906.0,6624.0,"3,162,920","2,582,185","580,735"
2013-08-02,8.0,금요일,27.4,25.2,29.3,7.5,2.0,50.0,80.4,14.1,...,44.0,10.0,18.0,3389.0,30199.0,27457.0,8091.0,"3,162,920","2,582,185","580,735"
2013-08-03,8.0,토요일,28.9,26.7,32.6,0.1,3.7,230.0,72.3,14.0,...,48.0,17.0,19.0,3935.0,35965.0,32984.0,9601.0,"3,162,920","2,582,185","580,735"
2013-08-04,8.0,일요일,28.3,26.4,31.3,4.5,2.4,250.0,76.4,14.0,...,46.0,16.0,14.0,3682.0,36219.0,31856.0,10519.0,"3,162,920","2,582,185","580,735"
2013-08-05,8.0,월요일,27.7,26.4,30.1,7.0,3.2,230.0,78.9,14.0,...,28.0,8.0,12.0,2645.0,27561.0,19661.0,6318.0,"3,162,920","2,582,185","580,735"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-08-27,8.0,화요일,25.7,22.2,29.3,20.0,1.0,290.0,71.1,13.2,...,8.0,8.0,10.0,2359.0,10162.0,8168.0,3832.0,"11,190,709","3,943,856","7,246,854"
2019-08-28,8.0,수요일,26.1,23.6,30.2,0.0,1.9,270.0,66.2,13.2,...,21.0,10.0,5.0,2341.0,10491.0,8694.0,3974.0,"11,190,709","3,943,856","7,246,854"
2019-08-29,8.0,목요일,23.4,20.1,26.4,36.9,2.2,160.0,77.1,13.1,...,18.0,11.0,10.0,2612.0,14513.0,9406.0,4543.0,"11,190,709","3,943,856","7,246,854"


In [50]:
merge_test

,월,요일,평균기온,최고기온,최저기온,일강수량,평균풍속,최다풍향,평균상대습도,가조시간,...,"중랑구, 50대, 여","중랑구, 60대, 남","중랑구, 60대, 여",족발/보쌈전문,중국음식,치킨,피자,합계,인터넷쇼핑,모바일쇼핑
key_0,,,,,,,,,,,,,,,,,,,,,
2013-08-01,8.0,목요일,28.0,24.5,32.1,0.0,1.7,50.0,67.3,14.1,...,55.0,NaN,6.0,2989.0,29310.0,21906.0,6624.0,"3,162,920","2,582,185","580,735"
2013-08-02,8.0,금요일,27.4,25.2,29.3,7.5,2.0,50.0,80.4,14.1,...,44.0,10.0,18.0,3389.0,30199.0,27457.0,8091.0,"3,162,920","2,582,185","580,735"
2013-08-03,8.0,토요일,28.9,26.7,32.6,0.1,3.7,230.0,72.3,14.0,...,48.0,17.0,19.0,3935.0,35965.0,32984.0,9601.0,"3,162,920","2,582,185","580,735"
2013-08-04,8.0,일요일,28.3,26.4,31.3,4.5,2.4,250.0,76.4,14.0,...,46.0,16.0,14.0,3682.0,36219.0,31856.0,10519.0,"3,162,920","2,582,185","580,735"
2013-08-05,8.0,월요일,27.7,26.4,30.1,7.0,3.2,230.0,78.9,14.0,...,28.0,8.0,12.0,2645.0,27561.0,19661.0,6318.0,"3,162,920","2,582,185","580,735"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-08-27,8.0,화요일,25.7,22.2,29.3,20.0,1.0,290.0,71.1,13.2,...,8.0,8.0,10.0,2359.0,10162.0,8168.0,3832.0,"11,190,709","3,943,856","7,246,854"
2019-08-28,8.0,수요일,26.1,23.6,30.2,0.0,1.9,270.0,66.2,13.2,...,21.0,10.0,5.0,2341.0,10491.0,8694.0,3974.0,"11,190,709","3,943,856","7,246,854"
2019-08-29,8.0,목요일,23.4,20.1,26.4,36.9,2.2,160.0,77.1,13.1,...,18.0,11.0,10.0,2612.0,14513.0,9406.0,4543.0,"11,190,709","3,943,856","7,246,854"


In [53]:
merge_test.to_pickle('data/CALLS_FINAL.pkl')

In [54]:
merge_test.to_csv('data/CALLS_FINAL.csv')